In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, r2_score, mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Dropout
from keras.utils import to_categorical
from keras.optimizers import SGD
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import mnist, cifar10


# Часть 1

## 1.1

### 1.1.2

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
def tanh(x):
    return np.tanh(x)

In [ ]:
def relu(x):
    return max(0.0, x)

In [ ]:
class Neuron:
    def __init__(self, weights, bias, activation_func):
        self.weights = weights
        self.bias = bias
        self.activation_func = activation_func

    def forward(self, inputs):
        total = np.dot(self.weights, inputs) + self.bias
        return self.activation_func(total)

In [ ]:
class NeuralNetwork:
    def __init__(self, activation_func):
        weights = np.array([0.5, 0.5, 0.5])
        bias = 0

        self.activation_func = activation_func

        self.h1 = Neuron(weights, bias, activation_func)
        self.h2 = Neuron(weights, bias, activation_func)
        self.h3 = Neuron(weights, bias, activation_func)
        self.o1 = Neuron(weights, bias, activation_func)

    def forward(self, x):
        out_h1 = self.h1.forward(x)
        out_h2 = self.h2.forward(x)
        out_h3 = self.h3.forward(x)
        out_o1 = self.o1.forward(np.array([out_h1, out_h2, out_h3]))
        return out_o1

In [ ]:
network_sigmoid = NeuralNetwork(
    activation_func=sigmoid
    )
x = np.array([2, 3, 4])

network_sigmoid.forward(x)

In [ ]:
network_tanh = NeuralNetwork(
    activation_func=tanh
    )
x = np.array([2, 3, 4])

network_tanh.forward(x)

In [ ]:
network_relu = NeuralNetwork(
    activation_func=relu
    )
x = np.array([2, 3, 4])

network_relu.forward(x)

## 1.2

### 1.2.1

In [ ]:
df_iris = pd.read_csv(
    'https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv'
    )

In [ ]:
df_iris['variety'] = df_iris['variety'].replace({
    'Setosa': 1, 'Versicolor': 2, 'Virginica': 3
})

In [ ]:
X_iris = df_iris.drop('variety', axis=1)
y_iris = df_iris['variety']

In [ ]:
X_train_iris, X_test_iris, \
y_train_iris, y_test_iris = \
    train_test_split(
        X_iris,
        y_iris,
        test_size=0.2,
        random_state=42
    )

In [ ]:
df_salary = pd.read_csv(
    'https://raw.githubusercontent.com/AnnaShestova/salary-years-simple-linear-regression/master/Salary_Data.csv'
)

In [ ]:
scaler_salary = StandardScaler()

In [ ]:
X_salary = df_salary[['YearsExperience']].values.reshape(-1, 1)
y_salary = df_salary['Salary']

In [ ]:
X_train_salary, X_test_salary, \
y_train_salary, y_test_salary = \
    train_test_split(
        X_salary,
        y_salary,
        test_size=0.2,
        random_state=42
    )

In [ ]:
X_train_salary_scaled = scaler_salary.fit_transform(X_train_salary)
X_test_salary_scaled = scaler_salary.transform(X_test_salary)

In [ ]:
param_grid_salary = {
    'activation': [
        'relu', 'tanh'
        ],
}

In [ ]:
clf_iris = MLPClassifier(
    hidden_layer_sizes=(100, 3),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42
)

In [ ]:
reg_salary = MLPRegressor(
    random_state=42,
)

In [ ]:
reg_salary.fit(X_train_salary_scaled, y_train_salary)

In [ ]:
clf_iris.fit(X_train_iris, y_train_iris)

In [ ]:
y_iris_pred = clf_iris.predict(X_test_iris)
acc_iris = accuracy_score(y_test_iris, y_iris_pred)
f1_iris = f1_score(y_test_iris, y_iris_pred, average='micro')

In [ ]:
y_salary_pred = reg_salary.predict(X_test_salary_scaled)
r2_salary = r2_score(y_test_salary, y_salary_pred)
mse_salary = mean_squared_error(y_test_salary, y_salary_pred)

In [ ]:
print(
    f"""Metrics for Iris Classifier:
    Accuracy: {acc_iris}
    F1 Score: {f1_iris}
    """
)

In [ ]:
print(
    f"""Metrics for Salary Regressor:
    R2 Score: {r2_salary}
    MSE: {mse_salary}
    """
)

# Часть 2

## 1.1

In [ ]:
X = np.random.rand(1000, 20)
y = np.random.randint(2, size=(1000,1))

In [ ]:
model = Sequential()
model.add(Dense(128, input_dim=20, activation='tanh'))
model.add(Dense(1, activation='softmax'))

In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy']
)

In [ ]:
model.fit(X, y, epochs=10, batch_size=32)

In [ ]:
loss, accuracy = model.evaluate(X, y)
print(
    f'Loss: {loss}',
    f'Accuracy: {accuracy}'
)

## 1.2

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(512, activation='relu'),
        layers.Dense(10, activation='softmax')
    ]
)

In [ ]:
model.compile(
    optimizer='rmsprop',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
train_images = train_images.reshape((60000, 28 * 28)).astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28)).astype('float32') / 255

In [ ]:
model.fit(train_images, train_labels, epochs=5, batch_size=64)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

In [ ]:
test_acc

## 1.3

In [ ]:
inputs = keras.Input(shape=(28, 28, 1))

In [ ]:
x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)

x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

In [ ]:
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

In [ ]:
model.compile(
    optimizer='rmsprop',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(train_images, train_labels, epochs=5, batch_size=64)

In [ ]:
test_loss, test_acc = model.evaluate(
    test_images, test_labels
)

In [ ]:
test_loss

In [ ]:
test_acc

## 1.4

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
class_names = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'
]

In [ ]:
batch_size = 32
nb_classes = len(class_names)
nb_epoch = 25
img_rows, img_cols = 32, 32
img_channels = 3

In [ ]:
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [ ]:
Y_train = to_categorical(y_train, nb_classes)
Y_test = to_categorical(y_test, nb_classes)

In [ ]:
model = Sequential()

In [ ]:
model.add(
    Conv2D(32, (3, 3)),
    activation='relu',
    padding='same'
)

In [ ]:
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

In [ ]:
sgd = SGD(
    learning_rate=0.01,
    decay=1e-6,
    momentum=0.9,
    nesterov=True
)

model.compile(
    loss='categorical_crossentropy',
    optimizer=sgd,
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    X_train, Y_train,
    batch_size=batch_size,
    epochs=nb_epoch,
    validation_split=0.1,
    shuffle=True,
    verbose=2
)

In [ ]:
scores = model.evaluate(X_test, Y_test, verbose=0)
scores